<a href="https://colab.research.google.com/github/ohilikeit/PUBG_dataset_project/blob/main/Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Load

In [ ]:
import os
import numpy as np
import warnings
warnings.filterwarnings(action='ignore') # 경고 메세지 숨김
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

raw_data = pd.read_csv('/content/drive/MyDrive/project1/data.csv')

# Preprocessing

## 1) 특성 제거
- Unnamed: 0 >> 불분명한 컬럼
- Id >> 개인 식별자

In [ ]:
raw_data = raw_data.drop(['Unnamed: 0', 'Id'], axis=1)

## 2) 이상데이터 삭제 및 파생변수 생성

In [ ]:
# 매치 킬 1등인데 최종순위 최하인 데이터 없애기
drop_index_1 = raw_data[(raw_data['killPlace'] == 1) & (raw_data['winPlacePerc'] == 0)].index
raw_data.drop(drop_index_1, inplace=True)
# 도보이동거리가 0인 데이터 삭제
drop_index_2 = raw_data[raw_data['walkDistance'] == 0].index
raw_data.drop(drop_index_2)
# 그룹 별 인원수 구하기
drop_index_3 = raw_data.groupby(['groupId']).size().to_frame('players_in_team')
raw_data = raw_data.merge(drop_index_3, how='left', on=['groupId'])
# 로드킬 5킬 이상 데이터 삭제
drop_index_4 = raw_data[raw_data['roadKills'] >= 5].index
raw_data.drop(drop_index_4, inplace=True)
# 힐링 아이템 사용횟수 40번 이상 데이터 삭제
drop_index_5 = raw_data[raw_data['heals'] >= 50].index
raw_data.drop(drop_index_5, inplace=True)
# 장거리 킬 중 800m 이상 데이터 삭제
drop_index_6 = raw_data[raw_data['longestKill'] >= 800].index
raw_data.drop(drop_index_6, inplace=True)
# 도보이동거리 10000m 이상 데이터 삭제
drop_index_7 = raw_data[raw_data['walkDistance'] >= 10000].index
raw_data.drop(drop_index_7, inplace=True)
# 수영이동거리 2000m 이상 데이터 삭제 
drop_index_8 = raw_data[raw_data['swimDistance'] >= 2000].index
raw_data.drop(drop_index_8, inplace=True)
# 무기습득횟수 50번 이상 데이터 삭제
drop_index_9 = raw_data[raw_data['weaponsAcquired'] >= 50].index
raw_data.drop(drop_index_9, inplace=True)
# 차량이동거리 0 인데 로드킬 한 데이터 삭제 
drop_index_10 = raw_data[(raw_data['roadKills'] >= 1) & (raw_data['rideDistance'] == 0)].index
raw_data.drop(drop_index_10, inplace=True)

In [ ]:
raw_data['HeadshotKills_perc'] = raw_data['headshotKills'] / (raw_data['kills'] + 1)  # 헤드샷킬 확률
raw_data['BattlePoints'] = raw_data['kills']*4 + raw_data['DBNOs']*2 + raw_data['assists']  # 전투포인트
raw_data['MedicalPoints'] = raw_data['boosts'] + raw_data['heals'] + raw_data['revives']    # 메디컬포인트
raw_data['TotalDistance'] = raw_data['rideDistance'] + raw_data['swimDistance'] + raw_data['walkDistance']  # 총 이동거리 
raw_data['Kills_per_time'] = raw_data['kills'] / (raw_data['matchDuration']+ 1)   # 초당 킬수
raw_data['Boost_per_time'] = raw_data['boosts'] / (raw_data['matchDuration'] + 1)  # 초당 도핑 횟수
raw_data['Teamwork'] = raw_data['assists'] + raw_data['revives']                       # 팀워크 정수 
raw_data['BattlePoints_per_distance'] = raw_data['BattlePoints'] / (raw_data['walkDistance'] + 1)   # 거리 당 전투포인트 
raw_data['boost_heals'] = raw_data['boosts'] + raw_data['heals']                       # 순수 힐링 
raw_data['After_combat'] = raw_data['damageDealt'] + raw_data['boost_heals']           # 전투 이후상황 
raw_data['assists_per_kill'] = raw_data['assists'] / (raw_data['kills'] + raw_data['assists'] + 1)  # 킬 대비 어시 수 
raw_data['road_kills_per_rideDistance'] = raw_data['roadKills'] / (raw_data['rideDistance'] + 1)  # 차량이동거리당 로드킬 비율 

raw_data[raw_data == np.Inf] = np.NaN    ## 무한대 값 처리하기 
raw_data[raw_data == np.NINF] = np.NaN

idx = raw_data[raw_data['BattlePoints_per_distance'].isna() == True].index
raw_data = raw_data.drop(idx, axis=0)
raw_data = raw_data.dropna(axis=0)

# 헤드샷킬 확률 100% 데이터 중 킬 수 5킬 이상인 경우 삭제(핵 사용 유저)
drop_index_10 = raw_data[(raw_data['HeadshotKills_perc'] == 1) & (raw_data['kills'] >= 5)].index
raw_data.drop(drop_index_10, inplace=True)

## 3) 정규화 및 범주형 변수 매핑

In [ ]:
df = raw_data.copy()

# winPlacePerc --> 실제 등수로 바꾸는 수식 
# df['target'] = round((1 - df['winPlacePerc']) * df['maxPlace'] + df['winPlacePerc'])

# 매치 타입(범주형 변수) 매핑
index = {"solo": 0,
         "duo": 1,
         "squad": 2,
         "solo-fpp": 3,
         "duo-fpp": 4,
         "squad-fpp": 5,
         "normal-squad-fpp": 6,
         "flaretpp": 7,
         "normal-duo-fpp": 8,
         "normal-squad": 9,
         "crashfpp": 10,
         "normal-solo-fpp": 11,
         "crashtpp": 12,
         "normal-solo": 13,
         "flarefpp": 14,
         "normal-duo": 15
         }
df['matchType_num'] = df['matchType'].map(lambda x: index[x])

# 사용할 feature 
cols = ['assists','boosts','damageDealt','DBNOs','headshotKills','matchType_num','heals', 'killPlace','kills','killStreaks','longestKill','matchDuration', 'killPoints', 'rankPoints', 'winPoints',
        'maxPlace','numGroups','revives','rideDistance','roadKills','walkDistance', 'swimDistance','teamKills','vehicleDestroys','HeadshotKills_perc','BattlePoints','MedicalPoints',
        'TotalDistance','Kills_per_time','Boost_per_time','Teamwork', 'BattlePoints_per_distance', 'After_combat','boost_heals', 'assists_per_kill', 'road_kills_per_rideDistance', 'winPlacePerc']

# 그룹별로 묶어서 정규화할 feature(팀으로 순위에 끼치는 영향이 큰 feature)
group_cols = ['assists','boosts','damageDealt','DBNOs','heals', 'killPlace','kills','killStreaks','revives','rideDistance','roadKills', 'walkDistance', 'swimDistance',
        'teamKills','vehicleDestroys','BattlePoints','MedicalPoints','TotalDistance','Kills_per_time','Boost_per_time','Teamwork','BattlePoints_per_distance',
        'road_kills_per_rideDistance','assists_per_kill',]
# 전체 데이터를 기준으로 정규화할 feature(개인 역량이 더 중요한 feature)
non_group_cols = ['headshotKills', 'matchType_num','longestKill','matchDuration', 'maxPlace', 'numGroups', 'HeadshotKills_perc','After_combat','boost_heals', 'killPoints', 'rankPoints', 'winPoints']

# 데이터셋 그룹화 후 평균치 사용 
data = df.groupby(['groupId','matchId']).mean()

# 그룹별 컬럼 정규화
def group_normalize(data):
    copyed = data.copy()
    mean = copyed.groupby('matchId').mean()
    std = copyed.groupby('matchId').std()
    std.fillna(1)  # 표준편차 Null 값은 1로 채우기 
    copyed[group_cols ]= (copyed[group_cols] - mean[group_cols]) / (std[group_cols] + 1e-8)
    
    return copyed

data = group_normalize(data)

# 전체 기준 정규화 
def whole_normalize(data):
    copyed = data.copy()
    idx = df[non_group_cols].describe().transpose()
    copyed[non_group_cols] = (copyed[non_group_cols] - idx['mean']) / idx['std']

    return copyed

data = whole_normalize(data)
data = data[cols]
data = data.fillna(0)

# train 셋과 test 셋으로 분리 
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, shuffle=True, random_state=42)

train.to_csv('/content/drive/MyDrive/project1/train.csv')
test.to_csv('/content/drive/MyDrive/project1/test.csv')